In [1]:
import os
import tensorflow as tf

# Directory paths
train_dir = "../data/chest_xray/new_train"
test_dir = "../data/chest_xray/new_test"
val_dir = "../data/chest_xray/new_val"

image_size = (64, 64)
batch_size = 64

def decode_img(img):
    # Decode the JPEG image and resize it
    img = tf.image.decode_jpeg(img, channels=1)  # Assuming grayscale images
    return tf.image.resize(img, image_size)

def process_path(file_path):
    # Extract label from the file path
    parts = tf.strings.split(file_path, os.sep)
    # Compare the second last part of the path with "PNEUMONIA"
    label = tf.where(parts[-2] == "PNEUMONIA", 1, 0)
    return file_path, label

def preprocess_image(file_path, label):
    # Read and preprocess the image
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

# Create a dataset of file paths
train_list_ds = tf.data.Dataset.list_files(os.path.join(train_dir, '*/*'), shuffle=False)
# Map the file paths to (image, label) pairs
train_labeled_ds = train_list_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
# Apply further preprocessing like decoding images
train_ds = train_labeled_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Applying cache, shuffle, batch, and prefetch to optimize the pipeline
# train_ds = train_ds.cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_list_ds = tf.data.Dataset.list_files(os.path.join(val_dir, '*/*'), shuffle=False)
val_labeled_ds = val_list_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_labeled_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
# val_ds = train_ds.cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

2024-02-22 12:10:31.596720: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras import layers, models

input_shape = image_size + (1,)
model = models.Sequential([
    layers.Flatten(input_shape=input_shape),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=10,  # Adjust the number of epochs based on your dataset and model complexity
    validation_data=val_ds)

Epoch 1/10


ValueError: in user code:

    File "/Users/ronlodetti/anaconda3/envs/tf-env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/ronlodetti/anaconda3/envs/tf-env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ronlodetti/anaconda3/envs/tf-env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/ronlodetti/anaconda3/envs/tf-env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Users/ronlodetti/anaconda3/envs/tf-env/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/ronlod